# CHAPTER 7
# Data Cleaning and Preparation
- Data preparation (loading, cleaning, transforming, rearranging) is reported to take up to 80% or more of an analyst's time.
- **pandas** along with buil-in Python features provide a high-level, flexible, and fast set of tools to manipulate data into the right form.

## Handling Missing Data
- Missing data occurs in many data analysis applications.
- **pandas** try to make working with missing data as painless as possible.
- For example all of the descriptive statistics on **pandas objects** exclude missing data by default.
- For numeric data, pandas uses the floating-point value **NaN (Not a Number)** to represent missing data - this is called a *sentinel value*.
- For other data types pandas uses **NA (not available)** to represent missing values. 
- In statistics applications, **NA data** may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). 
- When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Create a pandas Series of strings contaning one missing value
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
# Use isnull function to check for missing values
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# The built-in Python None value is also treated as NA in object arrays

# Replace the first element in string_data with None
string_data[0] = None

# Check for missing values
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

**TABLE**: NA handling methods

| Argument                  | Description |
| :---                  |    :----    |
|dropna| Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
|fillna| Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
|isnull| Return boolean values indicating which values are missing/NA.
|notnull| Negation of isnull.

### Filtering Out Missing Data
- You can filter out missing data by hand using **pandas.isnull** and boolean indexing.
- Or you can use the **dropna** function. 
- On a Series, **dropna** returns the Series with only the non-null data and index values.
- With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. **dropna** by default drops any row containing a missing value.

In [5]:
# Create a Series that contains missing values
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [6]:
# Use dropna to remove the missing values
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
# Create a DataFrame with missing values
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
# Use dropna to remove row with missing values
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [9]:
# Passing how='all' will only drop rows that are all NA
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [10]:
# Create a 4th column with all values NA
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
# To drop columns in the same way, pass axis=1
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


- Suppose you want to keep only rows containing a certain number of observations. 
- You can indicate this with the **thresh** argument for **dropna**.

In [12]:
# Create a DataFrame with 7 rows and 3 columns
df = pd.DataFrame(np.random.randn(7, 3))

# Replace first 4 values for column 1 with NA values
df.iloc[:4, 1] = np.nan

# Replace the first 2 values for column 2 with NA values
df.iloc[:2, 2] = np.nan

df

,0,1,2
0,-0.415559,NaN,NaN
1,-1.395907,NaN,NaN
2,-1.878527,NaN,-1.078754
3,0.289503,NaN,-1.164157
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


In [13]:
# If we use dropnan with no arguments all rows with at least 1 NA value will be filtered
df.dropna()

,0,1,2
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


In [14]:
# Using thresh argument we can indicate how many NA values in a row are allowed
df.dropna(thresh=2)

,0,1,2
2,-1.878527,NaN,-1.078754
3,0.289503,NaN,-1.164157
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


### Filling In Missing Data
- Rather than filtering out missing data you may want to fill in the “holes” in any number of ways. 
- For most purposes, the **fillna** method is the workhorse function to use.

In [15]:
# Calling fillna with a constant replaces missing values with that value
df.fillna(0)

,0,1,2
0,-0.415559,0.000000,0.000000
1,-1.395907,0.000000,0.000000
2,-1.878527,0.000000,-1.078754
3,0.289503,0.000000,-1.164157
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


In [16]:
# Calling fillna with a dict, you can use a different fill value for each column
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.415559,0.500000,0.000000
1,-1.395907,0.500000,0.000000
2,-1.878527,0.500000,-1.078754
3,0.289503,0.500000,-1.164157
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


In [17]:
# fillna returns a new object, but you can modify the existing object in-place
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.415559,0.000000,0.000000
1,-1.395907,0.000000,0.000000
2,-1.878527,0.000000,-1.078754
3,0.289503,0.000000,-1.164157
4,0.906449,-0.945880,-1.338671
5,0.008262,0.432657,0.055710
6,-0.254438,0.746407,0.464434


In [18]:
# Create a new DataFrame
df2 = pd.DataFrame(np.random.randn(6, 3))

# Insert some missing values
df2.iloc[2:, 1] = np.nan
df2.iloc[4:, 2] = np.nan

df2

,0,1,2
0,-0.239478,-0.328366,0.306886
1,1.894113,-1.469041,1.800257
2,-1.827413,NaN,-0.171031
3,-1.605733,NaN,-0.209629
4,0.614996,NaN,NaN
5,-2.256638,NaN,NaN


In [19]:
# The same interpolation methods available for reindexing can be used with fillna
df2.fillna(method='ffill')

# 'ffill' = forward fill method

,0,1,2
0,-0.239478,-0.328366,0.306886
1,1.894113,-1.469041,1.800257
2,-1.827413,-1.469041,-0.171031
3,-1.605733,-1.469041,-0.209629
4,0.614996,-1.469041,-0.209629
5,-2.256638,-1.469041,-0.209629


In [20]:
# Create a new Series with missing values
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

# For example you might pass the mean or median value of a Series
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

**TABLE**: *fillna* function arguments

| Argument                  | Description |
| :---                  |    :----    |
|value| Scalar value or dict-like object to use to fill missing values
|method| Interpolation; by default 'ffill' if function called with no other arguments
|axis| Axis to fill on; default axis=0
|inplace| Modify the calling object without producing a copy
|limit| For forward and backward filling, maximum number of consecutive periods to fill

## Data Transformation
### Removing Duplicates

In [21]:
# Create a DataFrame containing duplicates
df3 = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1, 1, 2, 3, 3, 4, 4]})
df3

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [22]:
# The DataFrame method duplicated returns a boolean Series indicating whether each
# row is a duplicate (has been observed in a previous row) or not

df3.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [23]:
# drop_duplicates returns a DataFrame where the duplicated array is False
df3.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [24]:
# Filter duplicates only based on the 'k1' column
df3.drop_duplicates(['k1'])

,k1,k2
0,one,1
1,two,1


In [25]:
# duplicated and drop_duplicates by default keep the first observed value combination
# Passing keep='last' will return the last one

df3.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


### Transforming Data Using a Function or Mapping
- For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame.

In [26]:
# Create a DataFrame with data about various kinds of meats
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [27]:
# Suppose you wanted to add a column indicating the type of animal that each food came from

# Create a dict to map each meat to the animal
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

# The map method on a Series accepts a function or dict-like object containing a mapping

In [28]:
# First we need to convert each value from 'food' column to lowercase using the str.lower
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [29]:
# Nowe we can use the Series map method to create an extra column called 'animal'
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [30]:
# Doing the same thing using a lambda function
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values
- Filling in missing data with the **fillna** method is a special case of more general value replacement. 
- **map** can be used to modify a subset of values in an object but **replace** provides a simpler and more flexible way to do so.

In [31]:
# Create a Series
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [32]:
# We can use replace to modify certail values
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [33]:
# replace multiple values at once
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [34]:
# To use a different replacement for each value, pass a list of substitutes
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes
- Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. 
- You can also modify the axes in-place without creating a new data structure.

In [35]:
# Create a DataFrame
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [36]:
# Like a Series, the axis indexes have a map method
data.index = data.index.map(lambda x: x[:4].upper())
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [37]:
# If you want to create a transformed version of a dataset without modifying the original, 
# a useful method is rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


### Discretization and Binning
- Continuous data is often discretized or otherwise separated into “bins” for analysis.

In [38]:
# Suppose you have data about a group of people and you want to group
# them into discrete age buckets

ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [39]:
# Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older
bins = [18, 25, 35, 60, 100]

In [40]:
# To create the actual bins for the data we can use the pandas.cut function
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

**pandas.cut**:
- The object pandas returns is a special **Categorical object**. 
- The output describes the bins computed by **pandas.cut** 
- It contains a categories array specifying the distinct **category names** along with a labeling for the ages data in the **codes attribute**.
- **pd.value_counts(cats)** are the bin counts for the result of **pandas.cut**
- You can change which side is closed by passing **right=False**.

In [41]:
# Check the codes
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [42]:
# Check the categories
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [43]:
# Check the bin count
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [44]:
# You can also pass your own bin names by passing a list or array to the labels option
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [45]:
# If you pass an integer number of bins to cut instead of explicit bin edges, it will compute 
# equal-length bins based on the minimum and maximum values in the data

data = np.random.randint(20, size=20)
data

array([ 4, 19, 16, 14,  8, 11,  6,  2,  6, 18, 11,  6, 19, 11, 14,  5,  4,
       19,  1, 19])

In [46]:
# Create 4 bins of equal-length
cats = pd.cut(data, 4, precision=0)
cats

[(1.0, 6.0], (14.0, 19.0], (14.0, 19.0], (10.0, 14.0], (6.0, 10.0], ..., (1.0, 6.0], (1.0, 6.0], (14.0, 19.0], (1.0, 6.0], (14.0, 19.0]]
Length: 20
Categories (4, interval[float64]): [(1.0, 6.0] < (6.0, 10.0] < (10.0, 14.0] < (14.0, 19.0]]

In [47]:
# Count the number of values in each bin
pd.value_counts(cats)

(14.0, 19.0]    6
(10.0, 14.0]    5
(1.0, 6.0]      5
(6.0, 10.0]     4
dtype: int64

- A closely related function, **qcut**, bins the data based on sample **quantiles**. 
- Depending on the distribution of the data, using cut will not usually result in each bin having the same number of data points. 
- Since qcut uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [48]:
# Create a sample of normally distributed numbers
data = np.random.randn(1000)

# Cut into quartiles
cats = pd.qcut(data, 4) 
cats

[(0.0176, 0.734], (0.0176, 0.734], (-3.5429999999999997, -0.677], (0.734, 3.15], (-0.677, 0.0176], ..., (-0.677, 0.0176], (-0.677, 0.0176], (-0.677, 0.0176], (-3.5429999999999997, -0.677], (0.0176, 0.734]]
Length: 1000
Categories (4, interval[float64]): [(-3.5429999999999997, -0.677] < (-0.677, 0.0176] < (0.0176, 0.734] < (0.734, 3.15]]

In [49]:
# Count the number of values in each bin
pd.value_counts(cats)

(0.734, 3.15]                    250
(0.0176, 0.734]                  250
(-0.677, 0.0176]                 250
(-3.5429999999999997, -0.677]    250
dtype: int64

### Detecting and Filtering Outliers
- Filtering or transforming outliers is largely a matter of applying array operations.

In [50]:
# Consider a DataFrame with some normally distributed data
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.049941,-0.027252,-0.021188,-0.007110
std,1.017280,1.010940,1.002563,1.043349
min,-3.543352,-2.860138,-2.947339,-3.498283
25%,-0.704054,-0.685124,-0.688526,-0.762600
50%,-0.053190,-0.024180,-0.047845,-0.003423
75%,0.654159,0.660167,0.623469,0.690601
max,3.451916,3.038879,2.948154,3.641397


In [51]:
data.head()

,0,1,2,3
0,-0.941420,-1.813778,-1.612194,-0.527527
1,0.279184,-1.725499,-0.527626,0.294463
2,-1.099327,-0.324381,0.554545,-0.846997
3,-0.304107,0.643425,0.729774,-0.030177
4,1.217377,2.132371,0.098258,1.112359


In [52]:
# Find values in one of the columns exceeding 3 in absolute value
col = data[2]
col[np.abs(col) > 3]

Series([], Name: 2, dtype: float64)

In [53]:
# To select all rows having a value exceeding 3 or –3, you can use the any method on a
# boolean DataFrame

data[(np.abs(data) > 3).any(1)]

,0,1,2,3
309,0.918895,-0.195733,-0.102543,3.641397
375,-3.543352,-0.462331,0.841483,-0.800795
425,-1.134037,3.038879,0.120837,-0.201184
752,-0.640595,-0.748268,-0.046682,-3.432917
829,0.881237,1.327890,0.187309,-3.498283
965,3.451916,-0.707110,-1.764049,-1.734953
984,3.061909,-0.369671,1.132398,1.471717


### Permutation and Random Sampling
- Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the **numpy.random.permutation** function. 
- Calling **permutation** with the length of the axis you want to permute produces an array of integers indicating the new ordering.

In [54]:
# Create a DataFrame
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [55]:
# Use the permutation function to create a sampler array
sampler = np.random.permutation(5)
sampler

array([0, 2, 3, 4, 1])

In [56]:
# Use the sampler array as input for take function = Return the elements in the given 
# positional indices along an axis
df.take(sampler)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
1,4,5,6,7


In [57]:
# To select a random subset without replacement, you can use the sample method
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
3,12,13,14,15


### Computing Indicator/Dummy Variables
- Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a **dummy** or **indicator matrix**. 
- If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. 
- pandas has a **get_dummies** function for doing this, though devising one yourself is not difficult.

In [58]:
# Create a DataFrame
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [59]:
# Use get_dummies on key column function to derive a matrix with 3 columns
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [60]:
# get_dummies has a prefix argument if you want to add a prefix to the columns in the 
# indicator DataFrame

# Create the dummy matrix with prefix
dummies = pd.get_dummies(df['key'], prefix='key')

# Join the dummy matrix with the original data
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


**EXAMPLE**: MovieLens 1M dataset

In [61]:
# Define name for the columns
mnames = ['movie_id', 'title', 'genres']

# Read the data from movies.dat file
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)

movies.head(10)

C:\Users\almne\Anaconda3\lib\site-packages\pandas\io\parsers.py:765: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [66]:
# Since a row in the DataFrame belongs to multiple categories
# Some wrangling is necessary to get the dummy matrix

# First, we extract the list of unique genres in the dataset
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))

genres = pd.unique(all_genres)

genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [67]:
# We start with a DataFrame of all zeros
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
# Rename the columns using the list of unq genres we created before
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
# Select first element as an example
gen = movies.genres[0]

# Split the text based on '|'
gen.split('|')

['Animation', "Children's", 'Comedy']

In [70]:
# Use the dummies.columns to compute the column indices for each genre
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [71]:
# We can use .iloc to set values based on these indices:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [72]:
# Combine this with movies
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[5]

movie_id                                 6
title                          Heat (1995)
genres               Action|Crime|Thriller
Genre_Animation                          0
Genre_Children's                         0
Genre_Comedy                             0
Genre_Adventure                          0
Genre_Fantasy                            0
Genre_Romance                            0
Genre_Drama                              0
Genre_Action                             1
Genre_Crime                              1
Genre_Thriller                           1
Genre_Horror                             0
Genre_Sci-Fi                             0
Genre_Documentary                        0
Genre_War                                0
Genre_Musical                            0
Genre_Mystery                            0
Genre_Film-Noir                          0
Genre_Western                            0
Name: 5, dtype: object

## String Manipulation
### String Object Methods
- In many string munging and scripting applications, built-in string methods are sufficient.

In [73]:
# As an example, a comma-separated string can be broken into pieces with split
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [74]:
# split is often combined with strip to trim whitespace (including line breaks)
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [75]:
# These substrings could be concatenated together with a two-colon delimiter using addition
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [76]:
# But this isn’t a practical generic method. A faster and more Pythonic way is to pass a
# list or tuple to the join method on the string '::'
'::'.join(pieces)

'a::b::guido'

In [77]:
# Using Python’s in keyword is the best way to detect a substring
'guido' in val

True

In [78]:
# Get the index of a certain substring
val.index(',')

# index raises an exception if the substring isn’t found

1

In [79]:
# You can also use find to get the index of asub string
val.find('a')

# find return -1 if a substring isn't found

0

In [80]:
# count returns the number of occurrences of a particular substring
val.count(',')

2

In [81]:
# replace will substitute occurrences of one pattern for another
# It is commonly used to delete patterns, too, by passing an empty string
val.replace(',', '')

'ab guido'

**TABLE**: Python built-in string methods

| Argument                  | Description |
| :---                  |    :----    |
|count| Return the number of non-overlapping occurrences of substring in the string.
|endswith| Returns True if string ends with suffix.
|startswith| Returns True if string starts with prefix.
|join| Use string as delimiter for concatenating a sequence of other strings.
|index |Return position of first character in substring if found in the string; raises ValueError if not found.
|find| Return position of first character of rst occurrence of substring in the string; like index, but returns –1 if not found.
|rfind| Return position of first character of last occurrence of substring in the string; returns –1 if not found.
|replace| Replace occurrences of string with another string.
|strip, rstrip, lstrip| Trim whitespace, including newlines; equivalent to x.strip() (and rstrip, lstrip, respectively) for each element.
|split| Break string into list of substrings using passed delimiter.
|lower| Convert alphabet characters to lowercase.
|upper| Convert alphabet characters to uppercase.
|casefold| Convert characters to lowercase, and convert any region-specific variable character  ombinations to a common comparable form.
|ljust, rjust| Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width.

### Regular Expressions
- Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. 
- A single expression, commonly called a **regex**, is a string formed according to the regular expression language. 
- Python’s built-in **re module** is responsible for applying regular expressions to strings.
- The **re module** functions fall into three categories: pattern matching, substitution, and splitting. 
- A **regex** describes a pattern to locate in the text, which can then be used for many purposes.

In [88]:
# Import the re module
import re

# Define a string with variable number of whitespace characters (tabs, spaces, and newlines)
text = "foo bar\t baz \tqux"
text

'foo bar\t baz \tqux'

In [86]:
# To remvoe the white spaces you can use the regex describing one or more whitespace 
# characters \s+ combined with re.split method
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [87]:
# If you want to get a list of all patterns matching the regex, you can use the findall method
re.findall('\s+', text)

[' ', '\t ', ' \t']

**Compile Regex**:
- When you call **re.split('\s+', text)**, the regular expression is first compiled, and then its split method is called on the passed text. 
- You can compile the regex yourself with **re.compile**, forming a reusable regex object.
- Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.

In [89]:
# Let’s consider a block of text and a regular expression capable of identifying 
# most email addresses

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [90]:
# Using findall on the text produces a list of the email addresses
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [91]:
# search returns a special match object for the first email address in the text
regex.search(text)

<re.Match object; span=(5, 20), match='dave@google.com'>

In [92]:
# regex.match returns None, as it only will match if the pattern 
# occurs at the start of the string
print(regex.match(text))

None


In [93]:
# sub will return a new string with occurrences of the pattern replaced by the
# a new string
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



**TABLE**: Regular expression methods

| Argument                  | Description |
| :---                  |    :----    |
|findall| Return all non-overlapping matching patterns in a string as a list
|finditer| Like findall, but returns an iterator
|match| Match pattern at start of string and optionally segment pattern components into groups; if the pattern matches, returns a match object, and otherwise None
|search| Scan string for match to pattern; returning a match object if so; unlike match, the match can be anywhere in the string as opposed to only at the beginning
|split| Break string into pieces at each occurrence of pattern
|sub, subn| Replace all (sub) or first n occurrences (subn) of pattern in string with replacement expression; use symbols \1, \2, ... to refer to match group elements in the replacement string

### Vectorized String Functions in pandas
- Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization. 
- To complicate matters, a column containing strings will sometimes have missing data.
- Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s **str** attribute.

In [94]:
# Create a Series of strings with NA values
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [95]:
# Check whether each email address has 'gmail' in it with str.contains
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [96]:
# Regular expressions can be used, too, along with any re options like IGNORECASE
data.str.findall('([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})', flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [97]:
# Match pattern at start of string - returns a match object
matches = data.str.match('([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})', flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [98]:
# You can slice strings using this syntax
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

**TABLE**: Partial listing of vectorized string methods

| Method                  | Description |
| :---                  |    :----    |
|cat| Concatenate strings element-wise with optional delimiter
|contains| Return boolean array if each string contains pattern/regex
|count| Count occurrences of pattern
|extract| Use a regular expression with groups to extract one or more strings from a Series of strings; the result will be a DataFrame with one column per group
|endswith| Equivalent to x.endswith(pattern) for each element
|startswith| Equivalent to x.startswith(pattern) for each element
|findall| Compute list of all occurrences of pattern/regex for each string
|get| Index into each element (retrieve i-th element)
|isalnum| Equivalent to built-in str.alnum
|isalpha| Equivalent to built-in str.isalpha
|isdecimal| Equivalent to built-in str.isdecimal
|isdigit| Equivalent to built-in str.isdigit
|islower| Equivalent to built-in str.islower
|isnumeric| Equivalent to built-in str.isnumeric
|isupper| Equivalent to built-in str.isupper
|join| Join strings in each element of the Series with passed separator
|len| Compute length of each string
|lower, upper| Convert cases; equivalent to x.lower() or x.upper() for each element
|match| Use re.match with the passed regular expression on each element, returning matched groups as list
|pad| Add whitespace to left, right, or both sides of strings
|center| Equivalent to pad(side='both')
|repeat| Duplicate values (e.g., s.str.repeat(3) is equivalent to x * 3 for each string)
|replace| Replace occurrences of pattern/regex with some other string
|slice| Slice each string in the Series
|split| Split strings on delimiter or regular expression
|strip| Trim whitespace from both sides, including newlines
|rstrip| Trim whitespace on right side
|lstrip| Trim whitespace on left side